In [23]:
import pandas as pd
import numpy as np

Dada la exitosa convocatoria de los Juegos Olímpicos de la Juventud por parte del público, sus organizadores
realizan distintos análisis para planificar las jornadas finales del certamen. Es por ello que cuentan con información en
los siguientes archivos csv:


eventos.csv (id_evento, fecha, id_locacion, nombre_evento, id_categoria_deportiva, cantidad_espectadores)

locacion.csv (id_locacion, nombre, capacidad, capacidad_extendida, sede, latitud, longitud)

categorias_deportivas.csv (id_categoria_deportiva, nombre, año_de_adopcion)


El primer archivo cuenta con información de los eventos, indicando la fecha (en formato “YYYY-mm-dd hh:mm:ss”), el
lugar donde ocurrió (id_locacion) y la cantidad de espectadores que asistieron. Además se aporta información sobre la
categoría deportiva a la cual pertenece el evento.

Por otro lado se tiene información sobre las distintas locaciones en la sedes del certamen en las que ocurrieron los
eventos. Contamos con información de su capacidad total de espectadores así como de su capacidad extendida (cuantos
asientos extras se pueden brindar sobre la capacidad de la locación).

### Resolvemos el punto (a)

a) Nombre de la sede que acumuló la mayor cantidad de espectadores en eventos durante el certamen del 14 al 15 de
octubre inclusive. Esto es de vital importancia para distribuir el merchandising oficial del evento, para las fechas finales.
(7 pts) 

In [24]:
eventos_data = {
        'fecha': ["2019-10-14 15:10:13","2019-10-15 15:10:13", "2019-10-16 15:10:13", "2019-09-14 15:10:13", "2019-10-14 16:10:13"],
        'id_locacion' : [1,1,2,3,4],
        'cantidad_espectadores': [1000, 1500, 3580, 115000, 383],
}

eventos = pd.DataFrame(eventos_data, columns = ['fecha', 'id_locacion', 'cantidad_espectadores'])

In [25]:
locacion_data = {               #cada locacion seria un estadio y la sede es la ciudad donde esta esa locacion
    'id_locacion': [1,2,3,4],
    'sede' : ['Nueva York', 'Nueva York', 'Miami', 'Nevada']
}

locacion = pd.DataFrame(locacion_data, columns = ['id_locacion', 'sede'])

In [26]:
eventos.head()

,fecha,id_locacion,cantidad_espectadores
0,2019-10-14 15:10:13,1,1000
1,2019-10-15 15:10:13,1,1500
2,2019-10-16 15:10:13,2,3580
3,2019-09-14 15:10:13,3,115000
4,2019-10-14 16:10:13,4,383


In [27]:
locacion.head()

,id_locacion,sede
0,1,Nueva York
1,2,Nueva York
2,3,Miami
3,4,Nevada


In [28]:
#eventos = pd.read_csv("eventos.csv")
eventos['fecha'] = pd.to_datetime(eventos['fecha'], errors = 'coerce') #si hay NaN's en las fechas evitamos que explote todo
eventos = eventos.loc[((eventos['fecha'].dt.day == 14) | (eventos['fecha'].dt.day == 15)) & (eventos['fecha'].dt.month == 10) ]

In [29]:
eventos.head()

,fecha,id_locacion,cantidad_espectadores
0,2019-10-14 15:10:13,1,1000
1,2019-10-15 15:10:13,1,1500
4,2019-10-14 16:10:13,4,383


In [30]:
cant_espectadores_por_locacion = eventos.groupby('id_locacion').agg({'cantidad_espectadores': 'sum'}).reset_index()

In [31]:
merge = cant_espectadores_por_locacion.merge(locacion, left_on = 'id_locacion', right_on = 'id_locacion', how = 'inner')

In [32]:
merge.nlargest(1, columns = 'cantidad_espectadores')

,id_locacion,cantidad_espectadores,sede
0,1,2500,Nueva York


### Resolvemos el punto (b)

b) Nombre del evento y nombre de la categoría deportiva de aquellos eventos cuya cantidad de espectadores superó la
capacidad de la locación, más allá de la capacidad extendida. Esto es de vital importancia para detectar problemas de
seguridad o si es necesario realizar algún cambio de locación. (8 pts) 

In [48]:
eventos_b_data = {
    'nombre_evento' : ['Evento1', 'Evento2', 'Evento3', 'Evento4'],
    'cantidad_espectadores' : [10000, 3000, 13000, 17000],
    'id_categoria_deportiva' : [1,1,3,1],
    'id_locacion' : [1,2,1,3]    
}

eventos_b = pd.DataFrame(eventos_b_data, columns = ['nombre_evento','cantidad_espectadores','id_categoria_deportiva','id_locacion'])

locacion_b_data = {
    'id_locacion': [1,2,3,4],
    'sede' : ['Nueva York', 'Nueva York', 'Miami', 'Nevada'],
    'capacidad': [5000, 13000, 9500, 24000],
    'capacidad_extendida' : [2000, 2000, 1500, 300] 
}

locacion_b = pd.DataFrame(locacion_b_data, columns = ['id_locacion', 'sede', 'capacidad', 'capacidad_extendida'])

categorias_deportivas = {
    'id_categoria_deportiva' : [1,3],
    'nombre' : ['futbol', 'tennis']
}

categorias = pd.DataFrame(categorias_deportivas, columns = ['id_categoria_deportiva', 'nombre'])

In [49]:
eventos_b.head()

,nombre_evento,cantidad_espectadores,id_categoria_deportiva,id_locacion
0,Evento1,10000,1,1
1,Evento2,3000,1,2
2,Evento3,13000,3,1
3,Evento4,17000,1,3


In [50]:
locacion_b.head()

,id_locacion,sede,capacidad,capacidad_extendida
0,1,Nueva York,5000,2000
1,2,Nueva York,13000,2000
2,3,Miami,9500,1500
3,4,Nevada,24000,300


In [51]:
categorias.head()

,id_categoria_deportiva,nombre
0,1,futbol
1,3,tennis


b) Nombre del evento y nombre de la categoría deportiva de aquellos eventos cuya cantidad de espectadores superó la
capacidad de la locación, más allá de la capacidad extendida. Esto es de vital importancia para detectar problemas de
seguridad o si es necesario realizar algún cambio de locación. (8 pts) 

In [52]:
locacion_b['capacidad_maxima'] = locacion_b['capacidad'] + locacion_b['capacidad_extendida']

In [61]:
merge_events_loc = eventos_b.merge(locacion_b, left_on = 'id_locacion', right_on = 'id_locacion', how = 'inner')
merge_events_loc

,nombre_evento,cantidad_espectadores,id_categoria_deportiva,id_locacion,sede,capacidad,capacidad_extendida,capacidad_maxima
0,Evento1,10000,1,1,Nueva York,5000,2000,7000
1,Evento3,13000,3,1,Nueva York,5000,2000,7000
2,Evento2,3000,1,2,Nueva York,13000,2000,15000
3,Evento4,17000,1,3,Miami,9500,1500,11000


In [62]:
merge_events_loc = merge_events_loc.loc[(merge_events_loc['cantidad_espectadores'] - merge_events_loc['capacidad_maxima']) > 0]
merge_events_loc

,nombre_evento,cantidad_espectadores,id_categoria_deportiva,id_locacion,sede,capacidad,capacidad_extendida,capacidad_maxima
0,Evento1,10000,1,1,Nueva York,5000,2000,7000
1,Evento3,13000,3,1,Nueva York,5000,2000,7000
3,Evento4,17000,1,3,Miami,9500,1500,11000


In [64]:
merge_events_loc = merge_events_loc_filtered[['nombre_evento', 'id_categoria_deportiva']]
merge_events_loc = merge_events_loc.merge\
(categorias, left_on = 'id_categoria_deportiva', right_on = 'id_categoria_deportiva', how = 'inner')
resul = merge_events_loc[['nombre_evento','nombre']]

In [65]:
resul

,nombre_evento,nombre
0,Evento1,futbol
1,Evento4,futbol
2,Evento3,tennis
